In [ ]:
import pandas as pd
import numpy as np
import yaml
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import datetime
import csv
import lightgbm as lgb
from sklearn.model_selection import train_test_split,KFold,cross_validate
import sklearn.datasets
import optuna 
CONFIG_FILE = '../configs/config.yaml'
with open(CONFIG_FILE) as file:
    yml = yaml.load(stream=file, Loader=yaml.SafeLoader)
MODEL_DIR_NAME = yml['SETTING']['MODEL_DIR_NAME']
feature_dir_name = yml['SETTING']['FEATURE_PATH']
target = 'y'

In [ ]:
features = [
'before_is_holiday'
,'client'
,'close'
,'day'
,'dayofyear_cos'
,'dayofyear_sin'
,'is_holiday'
,'month'
,'next_is_holiday'
,'price_am'
,'price_pm'
,'weekday'
,'year'
    ]

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from optuna.integration import lightgbm as lgb_optuna
from sklearn import preprocessing
from sklearn.model_selection import train_test_split,StratifiedKFold,cross_validate

pd.set_option('display.max_columns', None)

random_state = 1234
version = 'v1'

dfs = [pd.read_pickle(feature_dir_name + f'{f}_train.pkl') for f in features]
X = pd.concat(dfs, axis=1)
y = pd.read_pickle(feature_dir_name + target + '_train.pkl')
dfs = [pd.read_pickle(feature_dir_name + f'{f}_test.pkl') for f in features]
test_x = pd.concat(dfs, axis=1)
train_y = pd.read_pickle(feature_dir_name + target + '_train.pkl')
del dfs


X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.2, random_state=random_state)

In [ ]:
def build():
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)

    lgb_train = lgb_optuna.Dataset(X_train, y_train)

    lgbm_params = {
        'objective': 'regression',
        'metric': 'mae',
        'random_state':random_state,
        'verbosity': 0
    }

    tunecv = lgb_optuna.LightGBMTunerCV(
        lgbm_params,
        lgb_train,
        num_boost_round=20000,
        early_stopping_rounds=5000,
        seed = random_state,
        verbose_eval=2000,
        folds=kf
    )

    tunecv.run()

    print( 'Best score = ',tunecv.best_score)
    print( 'Best params= ',tunecv.best_params)

    return tunecv

tunecv = build()

In [ ]:
train_data = lgb.Dataset( X_train, y_train )
eval_data = lgb.Dataset(X_holdout, label=y_holdout, reference= train_data)
clf = lgb.train( tunecv.best_params, 
                train_data,
                valid_sets=eval_data,
                num_boost_round=20000,
                verbose_eval=0
               )
prediction_LG = clf.predict( test_x )

In [ ]:
submission = pd.DataFrame(prediction_LG)
id = pd.read_csv('../data/raw/sample_submission.csv',header=None)
dd = pd.concat([id,submission],axis=1)
now = datetime.datetime.now()
dd.to_csv('../data/submission/log_' + now.strftime('%Y%m%d_%H%M%S') + '.csv', header=False, index=False)